# Probable Maximum Precipitation (PMP) module

This example shows how to use the PMP module to compute the Probable Maximum Precipitation from climate change scenarios.  This module allows to compute the PMP on a grid (distributed) and for aggregated values of precipitation following the methodology shown by Clavet-Gaumont et al. (2017).

In [1]:
from pathlib import Path
from zipfile import ZipFile

import pooch
import xarray as xr
from lmoments3.distr import gev

from xhydro import pmp

## Open data 

This example uses a sample of 5-years and 2x2 grid cells from the CMIP model which can be accessed from the xhydro-testdata repository.

In [18]:
GITHUB_URL = "https://github.com/hydrologie/xhydro-testdata"
BRANCH_OR_COMMIT_HASH = "61e3b2b9a224d4dcc098c0acf3326cb13e22f441"

path_day_zip = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/pmp/CMIP.CCCma.CanESM5.historical.r1i1p1f1.day.gn.zarr.zip",
    known_hash="md5:191cffe11cacc303db697aa91d9be7ab",
)

path_fx_zip = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/pmp/CMIP.CCCma.CanESM5.historical.r1i1p1f1.fx.gn.zarr.zip",
    known_hash="md5:1257973a6f6047e6998c3430e3342534",
)

directory_to_extract_to_day = Path(
    path_day_zip
).parent  # To extract to the same directory as the zip file
with ZipFile(path_day_zip, "r") as zip_ref:
    zip_ref.extractall(directory_to_extract_to_day)

directory_to_extract_to_fx = Path(
    path_fx_zip
).parent  # To extract to the same directory as the zip file
with ZipFile(path_fx_zip, "r") as zip_ref:
    zip_ref.extractall(directory_to_extract_to_fx)

path_day_zarr = (
    str(directory_to_extract_to_day)
    + "/CMIP.CCCma.CanESM5.historical.r1i1p1f1.day.gn.zarr"
)
path_fx_zarr = (
    str(directory_to_extract_to_fx)
    + "/CMIP.CCCma.CanESM5.historical.r1i1p1f1.fx.gn.zarr"
)

For this example, the CMIP simulations on an daily scale  were used since it contains the variables necessary for the computing of the PMP:

ds_day
* pr --> Precipitation_flux  
* snw --> Snow water equivalent  
* hus --> Specific humidity 
* zg --> Geopotential height

ds_fx
* orog --> Surface altitude

In [34]:
ds_day = xr.open_zarr(path_day_zarr)
ds_fx = xr.open_zarr(path_fx_zarr)

## 1. PMP distributed

### 1.1. Major precipitations events (pe).

Sum of precipitation over 1, 2, 3 and 4 days and retention of the year's most intense storms.

The 10% most intense storms are retained to avoid overmaximizing small precipitation events.

In [4]:
pe = pmp.major_precipitation_events(
    ds_day.rf, acc_day=[1, 2, 3, 4], quantil=0.9, path=None
)

### 2.2 Calculating daily precipitable water (pw)

Integration of specific humidity for all pressure levels for 1, 2, 3 and 4 days. Then, only the pw values associated with the major precipitation events (pe) are selected.


In [5]:
pw = pmp.precipitable_water(ds_day, ds_fx, acc_day=[1, 2, 3, 4], path=None)
pw_events = pw.where(pe > 0)

###  2.3. Monthly 100-year return value of precipitable water (pw100)

According to Clavet-Gaumont et al. (2017), the pw100 is calculated using the Generalized Extreme Value (GEV) and limiting the maximum value for PW100 to be less than 20% larger than the largest value of the maximum PW values (mf=0.2).

In [6]:
pw100 = pmp.precipitable_water_100y(pw.sel(acc_day=1), dist=gev, mf=0.2, path=None)

### 2.4.  Maximization ratio (r) and Maximized rainfall (pmax)

In [7]:
r = pw100 / pw_events
pmax = r * pe

### 2.5. Definition of seasons

1) The start and end of winter consider a minimum number of days with snow of 14 and 90 days, respectively, to guarantee snow accumulation at the beginning of winter and that there is no thaw at the end.

2) The start and end of spring are defined 60 days before and 30 days after the end of winter.

In [8]:
mask = pmp.compute_spring_and_summer_mask(
    ds_day.snw,
    thresh="1 cm",
    window_wint_start=14,
    window_wint_end=90,
    spr_start=60,
    spr_end=30,
)

pmax_spring = (pmax * mask.mask_spring).rename("pmp_spring")
pmax_summer = (pmax * mask.mask_summer).rename("pmp_summer")

### 2.6. PMP

In [9]:
pmp_spring = pmax_spring.max("time")
pmp_summer = pmax_summer.max("time")

## 2. PMP with aggregated storm configurations

The spatial_average_storm_configurations function allows to spatially aggregate the storms following the different configurations shown in Clavet-Gaumont et al. (2017). Once aggregated, the calculation of the aggregated PMP follows the same steps shown above.

In [10]:
ds_day_agg = pmp.spatial_average_storm_configurations(ds_day.rf, 10).chunk(
    dict(time=-1)
)
pe_agg = pmp.major_precipitation_events(
    ds_day_agg.chunk(dict(conf=-1)), acc_day=[1, 2, 3, 4], quantil=0.9, path=None
)

# Precipitable water (pw) must first be calculated in a distributed manner and then spatially averaged to obtain the aggregated precipitable water.
pw_agg = pmp.spatial_average_storm_configurations(pw, 10)

pw_events_agg = pw_agg.where(pe_agg > 0)

pw100_agg = pmp.precipitable_water_100y(
    pw_agg.sel(acc_day=1), dist=gev, mf=0.2, path=None
)

r_agg = pw100_agg / pw_events_agg

pmax_agg = r_agg * pe_agg

# To create the spring and summer masks, the snow cover must first be spatially averaged (ds_snw_agg).
ds_snw_agg = pmp.spatial_average_storm_configurations(ds_day.snw, 10).chunk(
    dict(time=-1)
)
mask_agg = pmp.compute_spring_and_summer_mask(
    ds_snw_agg,
    thresh="1 cm",
    window_wint_start=14,
    window_wint_end=90,
    spr_start=60,
    spr_end=30,
)

pmax_spring_agg = (pmax_agg * mask_agg.mask_spring).rename("pmp_spring")
pmax_summer_agg = (pmax_agg * mask_agg.mask_summer).rename("pmp_summer")

pmp_spring_agg = pmax_spring_agg.max("time")
pmp_summer_agg = pmax_summer_agg.max("time")

## Results

In [11]:
pmp_spring_agg.sel(acc_day=2).values

array([60.67189856, 73.40169334, 72.17344421, 78.85401983, 68.91597628,
       63.72309869, 71.99999003, 56.71861083, 59.69289196, 54.32799065,
       68.69975861, 69.98327349, 56.00702449])

In [12]:
pmp_spring.sel(acc_day=2).values

array([[60.67189856, 73.40169334],
       [72.17344421, 78.85401983]])

In [13]:
pmp_summer_agg.sel(acc_day=2).values

array([132.16790102,  97.21404162, 113.25224885, 109.40165686,
       104.49179963, 102.05764168,  95.05785447,  87.06664538,
        98.31741597,  94.42134837,  88.28404808,  89.26246296,
        87.5754352 ])

In [14]:
pmp_summer.sel(acc_day=2).values

array([[132.16790102,  97.21404162],
       [113.25224885, 109.40165686]])